In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": "output"})
    
def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Junie, I live in South Korea", "Wow that's so cool!")

C:\Users\hojun\AppData\Local\Temp\ipykernel_37692\704134362.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1)
C:\Users\hojun\AppData\Local\Temp\ipykernel_37692\704134362.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [ ]:
add_message("South Korea is amazing", "I wish I could go there!")